In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from xgboost import XGBRegressor, XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics.pairwise import rbf_kernel
from scipy.stats import pearsonr
from IPython.display import display, Math, Markdown
from sklearn.model_selection import train_test_split


## 0. Introduction

Simulations in this document are for the approach outlined in draft 3.1. We generate data under settings such that sequential conditional exchangeability assumption holds with full set of covariates. We compute the population quantity of mean potential outcome under treatment path $\bar{A} = \bar{1}$ (1), the population AIPW $\psi$ based on (8), the incorrect population AIPW $\psi^*$ based on (11), their difference $\psi^*-\psi$ as listed (12). Moreover, we compute the basic ovb based on equation (13), simplified ovb based on equation (24),  the ovb bound based on (14) together with (15), and the ovb bound based on (14), (15), and (18).

## 1. Data Generating 

### 1.1 Description

\begin{align*}
U_0 &\sim \mathcal{N}(0, \sigma_{U_0}^2), \\[1em]
L_0 &= \beta_{L_0|U_0} U_0 + \epsilon_{L_0}, \quad \epsilon_{L_0} \sim \mathcal{N}(0, \sigma_{L_0}^2), \\[1em]
P(A_0 = 1 \mid L_0 = \ell_0, U_0 = u_0) &= \left[1 + \exp\{- (\beta_{A_0|L_0} \ell_0 + \beta_{A_0|U_0} u_0)\}\right]^{-1}, \\[1em]
L_1 &= \beta_{L_1|L_0} L_0 + \beta_{L_1|A_0} A_0 + \beta_{L_1|U_0} U_0 + \epsilon_{L_1}, \quad \epsilon_{L_1} \sim \mathcal{N}(0, \sigma_{L_1}^2), \\[1em]
P(A_1 = 1 \mid L_0 = \ell_0, L_1 = \ell_1, A_0 = a_0, U_0 = u_0) &= \left[1 + \exp\{- (\beta_{A_1|L_1} \ell_1 + \beta_{A_1|A_0} a_0 + \beta_{A_1|L_0} \ell_0 + \beta_{A_1|U_0} u_0)\}\right]^{-1}, \\[1em]
Y &= \gamma_{Y|L_0} L_0 + \gamma_{Y|L_1} L_1 + \gamma_{Y|A_0} A_0 + \gamma_{Y|A_1} A_1 + \gamma_{Y|U_0} U_0 + \epsilon_Y, \quad \epsilon_Y \sim \mathcal{N}(0, \sigma_Y^2).
\end{align*}

\begin{align*}
\beta_{L0} &= (\beta_{L_0|U_0} ) = 0.1, \\
\beta_{A_0} &= (\beta_{A_0|L_0}, \beta_{A_0|U_0}) = (0.5, 0.3),\\
\beta_{L_1} &= (\beta_{L_1|L_0}, \beta_{L_1|A_0}, \beta_{L_1|U_0}) = (0.6, 0.4, 0.2), \\
\beta_{A_1} &= (\beta_{A_1|L_1}, \beta_{A_1|A_0}, \beta_{A_1|L_0}, \beta_{A_1|U_0}) = (0.4, 0.2, 0.3, 0.2), \\
\gamma_{Y} &= (\gamma_{Y|L_0}, \gamma_{Y|L_1} L_1, \gamma_{Y|A_0} A_0,  \gamma_{Y|A_1} , \gamma_{Y|U_0} ) = (1.2, 0.8, 0.5, 0.7, 0.5),\\
\sigma_{U_0} &= 1,\\
\sigma_{L_0} &= 1,\\
\sigma_{L_1} &= 1,\\
\sigma_Y &= 1.
\end{align*}



In [2]:
# Setting seed
np.random.seed(43)

# Large sample size
n = 20000000

# Define parameters for the data-generating process
params = {
    "U0_std": 1.0,          # Standard deviation for U_0
    "L0_U0_coeff": 0.1,     # Reduced coefficient for U_0 in L_0 <--
    "L0_noise_std": 1.0,    # Standard deviation for noise in L_0
    "A0_L0_coeff": 0.5,     # Coefficient for L_0 in A_0
    "A0_U0_coeff": 0.3,     # Reduced coefficient for U_0 in A_0 <--
    "L1_L0_coeff": 0.6,     # Coefficient for L_0 in L_1
    "L1_A0_coeff": 0.4,     # Coefficient for A_0 in L_1
    "L1_U0_coeff": 0.2,     # Reduced coefficient for U_0 in L_1 <--
    "L1_noise_std": 1.0,    # Standard deviation for noise in L_1
    "A1_L1_coeff": 0.4,     # Coefficient for L_1 in A_1
    "A1_A0_coeff": 0.2,     # Coefficient for A_0 in A_1
    "A1_L0_coeff": 0.3,     # Coefficient for L_0 in A_1
    "A1_U0_coeff": 0.2,     # Reduced coefficient for U_0 in A_1 <--
    "Y_L0_coeff": 1.2,      # Coefficient for L_0 in Y
    "Y_L1_coeff": 0.8,      # Coefficient for L_1 in Y
    "Y_A0_coeff": 0.5,      # Coefficient for A_0 in Y
    "Y_A1_coeff": 0.7,      # Coefficient for A_1 in Y
    "Y_U0_coeff": 0.5,      # Reduced coefficient for U_0 in Y <--
    "Y_noise_std": 1.0      # Standard deviation for noise in Y
}

# Step 1: Generate U0 (baseline unobserved covariates)
U_0 = np.random.normal(0, params["U0_std"], n)  # U_0 ~ Normal(0, std)

# Step 2: Generate L0 (baseline observed covariates) as a function of U0
L_0 = params["L0_U0_coeff"] * U_0 + np.random.normal(0, params["L0_noise_std"], n)

# Step 3: Generate A0 (treatment at time 0) based on L0 and U0
p_A0 = 1 / (1 + np.exp(-(params["A0_L0_coeff"] * L_0 + params["A0_U0_coeff"] * U_0)))
A_0 = np.random.binomial(1, p_A0, n)

# Step 4: Generate L1 (covariates at time 1) based on L0, U0, and A0
L_1 = (params["L1_L0_coeff"] * L_0 +
       params["L1_A0_coeff"] * A_0 +
       params["L1_U0_coeff"] * U_0 +
       np.random.normal(0, params["L1_noise_std"], n))

# Step 5: Generate A1 (treatment at time 1) based on L0, U0, L1, and A0
p_A1 = 1 / (1 + np.exp(-(params["A1_L1_coeff"] * L_1 +
                        params["A1_A0_coeff"] * A_0 +
                        params["A1_L0_coeff"] * L_0 +
                        params["A1_U0_coeff"] * U_0)))
A_1 = np.random.binomial(1, p_A1, n)

# Step 6: Generate observed outcome Y based on L0, U0, A0, A1, and L1
Y = (params["Y_L0_coeff"] * L_0 +
     params["Y_L1_coeff"] * L_1 +
     params["Y_A0_coeff"] * A_0 +
     params["Y_A1_coeff"] * A_1 +
     params["Y_U0_coeff"] * U_0 +
     np.random.normal(0, params["Y_noise_std"], n))


In [3]:
# Step 7: Regenerate L1 under the intervention A0=1
# L1_bar1: covariates at time 1 assuming A0=1
L1_bar1 = (params["L1_L0_coeff"] * L_0 +
           params["L1_A0_coeff"] * 1 +  # Intervention A0=1
           params["L1_U0_coeff"] * U_0 +
           np.random.normal(0, params["L1_noise_std"], n))

# Y_bar1: potential outcome under treatment path (A0=1, A1=1)
Y_bar1 = (params["Y_L0_coeff"] * L_0 +
          params["Y_L1_coeff"] * L1_bar1 +
          params["Y_A0_coeff"] * 1 +  # Intervention A0=1
          params["Y_A1_coeff"] * 1 +  # Intervention A1=1
          params["Y_U0_coeff"] * U_0 +
          np.random.normal(0, params["Y_noise_std"], n))

In [4]:
# Combine data into a DataFrame
data = pd.DataFrame({
    "L_0": L_0,
    "U_0": U_0,
    "A_0": A_0,
    "L_1": L_1,
    "A_1": A_1,
    "Y": Y,
    "L1_bar1": L1_bar1,
    "Y_bar1": Y_bar1
})

print(data.shape)

(20000000, 8)



### 1.2 Check sequential conditional exchangeability holds with full set of covariates via ROC AUC

In [5]:
# Exploratory Checks Using ROC AUC via prediction model XGBoost 
def roc_aud_conditional_independence_test(X, y, additional_var=None):
    """Test conditional independence using XGBoost and ROC AUC."""
    model = XGBClassifier(eval_metric="logloss")
    model.fit(X, y)
    baseline_roc_auc = roc_auc_score(y, model.predict_proba(X)[:, 1])

    if additional_var is not None:
        X_with_additional = np.column_stack((X, additional_var))
        model_with_additional = XGBClassifier(eval_metric="logloss")
        model_with_additional.fit(X_with_additional, y)
        additional_roc_auc = roc_auc_score(y, model_with_additional.predict_proba(X_with_additional)[:, 1])
        return baseline_roc_auc, additional_roc_auc
    else:
        return baseline_roc_auc



In [6]:
# Test 1: Y^{\bar{1}} ⫫ A0 | L0, U0
X = data[['L_0', 'U_0']].values
y = data['A_0'].values
baseline_auc, auc_with_y = roc_aud_conditional_independence_test(X, y, data['Y_bar1'].values)
print(f"Test 1 (with U0): Baseline AUC={baseline_auc}, AUC with Y_bar1={auc_with_y}")


KeyboardInterrupt: 

In [33]:
# Test 2: Y^{\bar{1}} ⫫ A0 | L0
X = data[['L_0']].values
y = data['A_0'].values
baseline_auc, auc_with_y = roc_aud_conditional_independence_test(X, y, data['Y_bar1'].values)
print(f"Test 2 (without U0): Baseline AUC={baseline_auc}, AUC with Y_bar1={auc_with_y}")


Test 2 (without U0): Baseline AUC=0.7275974381460293, AUC with Y_bar1=0.8237608423671091


### 1.3 Check sequential conditional exchangeability is violated with only observed set of covaraites via ROC AUC 

In [34]:
# Test 3: Y^{\bar{1}} ⫫ A1 | A0, L0, U0, L1
X = data[['A_0', 'L_0', 'U_0', 'L_1']].values
y = data['A_1'].values
baseline_auc, auc_with_y = roc_aud_conditional_independence_test(X, y, data['Y_bar1'].values)
print(f"Test 3 (with U0): Baseline AUC={baseline_auc}, AUC with Y_bar1={auc_with_y}")


Test 3 (with U0): Baseline AUC=0.852919989085951, AUC with Y_bar1=0.8531379646456854


In [35]:
# Test 4: Y^{\bar{1}} ⫫ A1 | A0, L0, L_1
X = data[['A_0', 'L_0', 'L_1']].values
y = data['A_1'].values
baseline_auc, auc_with_y = roc_aud_conditional_independence_test(X, y, data['Y_bar1'].values)
print(f"Test 4 (without U0): Baseline AUC={baseline_auc}, AUC with Y_bar1={auc_with_y}")


Test 4 (without U0): Baseline AUC=0.8278589512706339, AUC with Y_bar1=0.8443723662717287


### 1.4 Check sequential conditional exchangeability holds with full set of covariates via statistical test

In [12]:
def kernel_conditional_independence_test(X, Y, Z, gamma=1.0, kernel='rbf', threshold=0.05):
    """
    Perform a kernel-based conditional independence test.

    Parameters:
    - X: np.array, the first variable to test for independence
    - Y: np.array, the second variable to test for independence
    - Z: np.array, the conditioning set
    - gamma: float, bandwidth for RBF kernel
    - kernel: str, kernel type ('rbf' only in this implementation)
    - threshold: float, significance threshold for rejecting the null

    Returns:
    - p_value: float, p-value for the conditional independence test
    - decision: bool, True if conditional independence is rejected (dependent)
    """

    # Ensure inputs are numpy arrays
    X = np.array(X).reshape(-1, 1)
    Y = np.array(Y).reshape(-1, 1)
    Z = np.array(Z)

    # Compute kernels
    if kernel == 'rbf':
        K_X = rbf_kernel(X, gamma=gamma)
        K_Y = rbf_kernel(Y, gamma=gamma)
        K_Z = rbf_kernel(Z, gamma=gamma)
    else:
        raise ValueError("Unsupported kernel. Only 'rbf' is implemented.")

    # Compute residual kernels
    K_X_given_Z = K_X - K_Z @ np.linalg.pinv(K_Z) @ K_X
    K_Y_given_Z = K_Y - K_Z @ np.linalg.pinv(K_Z) @ K_Y

    # Compute the test statistic (correlation of residual kernels)
    test_statistic, _ = pearsonr(K_X_given_Z.ravel(), K_Y_given_Z.ravel())

    # For simplicity, compute a pseudo p-value based on absolute correlation
    p_value = 1 - abs(test_statistic)  # Lower value = more significant dependency

    # Decide if conditional independence is rejected
    decision = p_value < threshold

    return p_value, decision

In [ ]:
# Test 1  whether Y^{\bar{1}} is conditionally independent of A0 given L0, U0
X = data['Y_bar1'].values  # Variable to test
Y = data['A_0'].values     # Variable to test
Z = data[['L_0', 'U_0']].values  # Conditioning set

p_value, decision = kernel_conditional_independence_test(X, Y, Z, gamma=1.0)
print(f"Test 1 (Y_bar1 ⫫ A0 | L0, U0): p-value={p_value}, Reject CI={decision}")

In [ ]:
# Test 2 whether Y^{\bar{1}} is conditionally independent of A0 given L0
Z_without_U = data[['L_0']].values  # Conditioning set without U0
p_value, decision = kernel_conditional_independence_test(X, Y, Z_without_U, gamma=1.0)
print(f"Test 2 (Y_bar1 ⫫ A0 | L0): p-value={p_value}, Reject CI={decision}")

### 1.5 Check sequential conditional exchangeability is violated with only observed set of covaraites via statistical test


In [ ]:
# Test 3 whether Y^{\bar{1}} is conditionally independent of A1 given A0, L0, U0, L1
X = data['Y_bar1'].values  # Variable to test
Y = data['A_1'].values     # Variable to test
Z = data[['A_0', 'L_0', 'U_0', 'L_1']].values  # Conditioning set
p_value, decision = kernel_conditional_independence_test(X, Y, Z, gamma=1.0)
print(f"Test 3 (Y_bar1 ⫫ A1 | A0, L0, U0, L1): p-value={p_value}, Reject CI={decision}")


In [ ]:
# Test 4 whether Y^{\bar{1}} is conditionally independent of A1 given A0, L0, L1
Z_without_U = data[['A_0', 'L_0', 'L_1']].values  # Conditioning set without U0
p_value, decision = kernel_conditional_independence_test(X, Y, Z_without_U, gamma=1.0)
print(f"Test 4 (Y_bar1 ⫫ A1 | A0, L0, L1): p-value={p_value}, Reject CI={decision}")


## 2. Compute population quantities


In [5]:
# Restrict to certain subset, follow the math expression

# Split the data into training and prediction sets
train_data = data[:10000000]   # First 1000 rows for training
predict_data = data[10000000:].copy() # Last 1000 rows for prediction

# # Step 1: estimate b1_true and attach to predict_data
# train_subset_a0a1 = train_data[(train_data['A_0'] == 1) & (train_data['A_1'] == 1)]
# X_train1 = train_subset_a0a1[['L_0', 'L_1', 'U_0']]
# Y_train1 = train_subset_a0a1['Y']
# model1 = LinearRegression()
# model1.fit(X_train1, Y_train1)
# X_predict1 = predict_data[['L_0', 'L_1', 'U_0']]
# predict_data = predict_data.copy()  # Create an explicit copy to avoid SettingWithCopyWarning
# predict_data.loc[:, 'b1_true'] = model1.predict(X_predict1)  # Use .loc for explicit assignment

# Step 1: get b1_true from the data generating process directly
predict_data['b1_true'] = (params["Y_L0_coeff"] * predict_data['L_0'] +
                           params["Y_L1_coeff"] * predict_data['L_1'] +
                           params["Y_A0_coeff"] * 1 +  # Set A0=1
                           params["Y_A1_coeff"] * 1 +  # Set A1=1
                           params["Y_U0_coeff"] * predict_data['U_0'])

# Step 2: estimate b1_short and attach to predict_data
train_subset_a0a1 = train_data[(train_data['A_0'] == 1) & (train_data['A_1'] == 1)]
X_train2 = train_subset_a0a1[['L_0', 'L_1']]
Y_train2 = train_subset_a0a1['Y']
model2 = LinearRegression()
model2.fit(X_train2, Y_train2)
X_predict2 = predict_data[['L_0', 'L_1']]
# No additional .copy() needed here as we already created one in Step 1
predict_data.loc[:, 'b1_short'] = model2.predict(X_predict2)  # Use .loc for consistency

# # Step 5: estimate pi1_true
# train_subset_a0 = train_data[train_data['A_0'] == 1]
# X_train5 = train_subset_a0[['L_0', 'L_1', 'U_0']]
# Y_train5 = train_subset_a0['A_1']
# model5 = LogisticRegression()
# model5.fit(X_train5, Y_train5)
# X_predict5 = predict_data[['L_0', 'L_1', 'U_0']]
# predict_data.loc[:, 'pi1_true'] = model5.predict_proba(X_predict5)[:, 1]

# Step 5: get pi1_true from the data generating process directly
predict_data.loc[:, 'pi1_true'] = 1 / (1 + np.exp(-(params["A1_L1_coeff"] * predict_data['L_1'] +
                                            params["A1_A0_coeff"] * 1 +  # Set A0 = 1
                                            params["A1_L0_coeff"] * predict_data['L_0'] +
                                            params["A1_U0_coeff"] * predict_data['U_0'])))

# Step 6: estimate pi1_short
train_subset_a0 = train_data[train_data['A_0'] == 1]
X_train6 = train_subset_a0[['L_0', 'L_1']]
Y_train6 = train_subset_a0['A_1']
# model6 = LogisticRegression()
# model6 = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=43)  # Nonparametric model
model6 = XGBClassifier(
    n_estimators=100,     # Number of trees
    max_depth=5,          # Maximum depth of a tree
    learning_rate=0.1,    # Learning rate (eta)
    subsample=0.8,        # Subsample ratio of training data
    colsample_bytree=0.8, # Subsample ratio of columns when constructing each tree
    random_state=43       # Random seed for reproducibility
)
model6.fit(X_train6, Y_train6)
X_predict6 = predict_data[['L_0', 'L_1']]
predict_data.loc[:, 'pi1_short'] = model6.predict_proba(X_predict6)[:, 1]

# # Step 7: estimate pi0_true
# X_train7 = train_data[['L_0', 'U_0']]
# Y_train7 = train_data['A_0']
# model7 = LogisticRegression()
# model7.fit(X_train7, Y_train7)
# X_predict7 = predict_data[['L_0', 'U_0']]
# predict_data.loc[:, 'pi0_true'] = model7.predict_proba(X_predict7)[:, 1]

# Step 7: get pi0_true from the data generating process directly
predict_data.loc[:, 'pi0_true'] = 1 / (1 + np.exp(-(params["A0_L0_coeff"] * predict_data['L_0'] +
                                            params["A0_U0_coeff"] * predict_data['U_0'])))

# Step 8: estimate pi0_short
X_train8 = train_data[['L_0']]
Y_train8 = train_data['A_0']
# model8 = LogisticRegression()
# model8 = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=43)  # Nonparametric model
model8 = XGBClassifier(
    n_estimators=100,     # Number of trees
    max_depth=5,          # Maximum depth of a tree
    learning_rate=0.1,    # Learning rate (eta)
    subsample=0.8,        # Subsample ratio of training data
    colsample_bytree=0.8, # Subsample ratio of columns when constructing each tree
    random_state=43       # Random seed for reproducibility
)
model8.fit(X_train8, Y_train8)
X_predict8 = predict_data[['L_0']]
predict_data.loc[:, 'pi0_short'] = model8.predict_proba(X_predict8)[:, 1]

# Step 3: estimate b0_true
predict_train, predict_test = train_test_split(predict_data, test_size=0.5, random_state=42)
predict_test = predict_test.copy()
predict_train_subset = predict_train[predict_train['A_0'] == 1].copy()
X_train3 = predict_train_subset[['L_0', 'U_0']]
Y_train3 = predict_train_subset['b1_true']
model3 = LinearRegression()
model3.fit(X_train3, Y_train3)
X_test3 = predict_test[['L_0', 'U_0']]
predict_test.loc[:, 'b0_true'] = model3.predict(X_test3)  

# Step 10: estimate ite_b1b0_short_true
X_train10 = predict_train_subset[['L_0', 'U_0']]
Y_train10 = predict_train_subset['b1_short']
model10 = LinearRegression()
model10.fit(X_train10, Y_train10)
X_test10 = predict_test[['L_0', 'U_0']]
predict_test.loc[:, 'ite_b1b0_short_true'] = model10.predict(X_test10)  

# Step 4: estimate b0_short
X_train4 = predict_train_subset[['L_0']]
Y_train4 = predict_train_subset['b1_short']
model4 = LinearRegression()
model4.fit(X_train4, Y_train4)
X_test4 = predict_test[['L_0']]
predict_test.loc[:, 'b0_short'] = model4.predict(X_test4)

In [6]:
# Compute important quantities 
a0pi0_short = predict_test['A_0'] / predict_test['pi0_short']
a1pi1_short = predict_test['A_1'] / predict_test['pi1_short']
a0pi0_true = predict_test['A_0'] / predict_test['pi0_true']
a1pi1_true = predict_test['A_1'] / predict_test['pi1_true']

diff_a0pi0 = a0pi0_true - a0pi0_short
diff_a1pi1 = a1pi1_true - a1pi1_short

diff_b0 = predict_test['b0_short'] - predict_test['b0_true']
diff_b1 = predict_test['b1_short'] - predict_test['b1_true']


### 2.1 Compute the population quantity of mean potential outcome under treatment path $\bar{A} = \bar{1}$ (1)


In [7]:
mean_Y_bar1 = np.mean(Y_bar1)
display(Math(r"E[Y^{{\bar{{1}}}}] = {:.4f}".format(mean_Y_bar1)))

<IPython.core.display.Math object>

### 2.2 Compute the population AIPW $\psi$ based on (8)



In [10]:
# Calculate psi_true
psi_true = np.mean(predict_test['b0_true'] - a0pi0_true * predict_test['b0_true']
            - a0pi0_true * a1pi1_true * predict_test['b1_true']
            + a0pi0_true * predict_test['b1_true']
            + a0pi0_true * a1pi1_true * predict_test['Y'])

display(Math(r"\psi = {:.4f}".format(psi_true)))


<IPython.core.display.Math object>

### 2.3 Compute the incorrect population AIPW $\psi^*$ based on (11)

In [12]:
# Calculate psi_short
psi_short = np.mean(predict_test['b0_short'] - a0pi0_short * predict_test['b0_short']
            - a0pi0_short * a1pi1_short * predict_test['b1_short']
            + a0pi0_short * predict_test['b1_short']
            + a0pi0_short * a1pi1_short * predict_test['Y'])

display(Math(r"\psi^* = {:.4f}".format(psi_short)))


<IPython.core.display.Math object>

### 2.4 Compute their difference $\psi^*-\psi$ as listed (12)

In [13]:
diff_psi = psi_short - psi_true
display(Math(r"\psi^* - \psi = {:.4f}".format(diff_psi)))


<IPython.core.display.Math object>

### 2.5 Compute the basic ovb based on equation (13)

In [14]:
ovb_basic = np.mean(diff_b0 * diff_a0pi0 + a0pi0_short * diff_b1 * diff_a1pi1)
print(f"Basic OVB Formula = {ovb_basic:.4f}")

Basic OVB Formula = 0.1339


### 2.6 Compute simplified ovb based on equation (24), (25)

In [15]:
K1 = np.mean((predict_test['b0_short']- predict_test['ite_b1b0_short_true']) * diff_a0pi0)
K2 = np.mean((predict_test['ite_b1b0_short_true'] - predict_test['b0_true']) * diff_a0pi0)
K3 = np.mean(diff_b1 * (a1pi1_true * a0pi0_true - a1pi1_short * a0pi0_short))
K4 = np.mean(diff_b1 * a1pi1_true * (a0pi0_short - a0pi0_true))
print(np.mean(a1pi1_true * a0pi0_true - a1pi1_short * a0pi0_short))

sum_K2_K4 = K2 + K4
sum_K1_K2_K3_K4 = K1 + K2 + K3 + K4
sum_K1_K3 = K1 + K3
sum_B1_B2 = K1 + K3
print(f"K2 + K4 = {sum_K2_K4:.4f}")
print(f"K1 + K2 + K3 + K4 = {sum_K1_K2_K3_K4:.4f}")
print(f"K1 + K3 = {sum_K1_K3:.4f}")
print(f"B1 + B2 = {sum_B1_B2:.4f}")


-0.00013668066535688634
K2 + K4 = 0.0001
K1 + K2 + K3 + K4 = 0.1339
K1 + K3 = 0.1338
B1 + B2 = 0.1338


### 2.7 Compute the ovb bound based on (14) together with (15)

In [16]:
# Compute upper bound of B1_square and B2_square
B11_square = np.mean((predict_test['b0_short']- predict_test['ite_b1b0_short_true']) ** 2)
B12_square = np.mean(diff_a0pi0 ** 2)
B21_square = np.mean(diff_b1 ** 2)
B22_square = np.mean((a1pi1_true * a0pi0_true - a1pi1_short * a0pi0_short) ** 2)
print(B11_square, B12_square, B21_square, B22_square)  # variation 160.417 is huge 
print(np.mean(a1pi1_true * a0pi0_true - a1pi1_short * a0pi0_short))  # mean is 0.0068

B1_square_ub = B11_square * B12_square
B2_square_ub = B21_square * B22_square
print(B1_square_ub, B2_square_ub)

B1_ub = np.sqrt(B1_square_ub)
B2_ub = np.sqrt(B2_square_ub)
print(B1_ub, B2_ub)

ovb_ub = B1_ub + B2_ub
display(Math(r"|\psi^* - \psi| \le |B_1| + |B_2| \le {:.4f}".format(ovb_ub)))

0.031278353527278355 0.05959231380136106 0.24513946477352097 0.3764737921256619
-0.00013668066535688634
0.0018639494585874804 0.09228858390294255
0.04317348096444715 0.30379036176768764


<IPython.core.display.Math object>

### 2.8  Compute the ovb bound based on (14), (15), and (18).

In [17]:
C1b1_square = (np.mean((predict_test['b0_short'] - predict_test['ite_b1b0_short_true']) ** 2) / 
               np.mean((predict_test['b1_short'] - predict_test['ite_b1b0_short_true']) ** 2))
S1_square = np.mean((predict_test['b1_short'] - predict_test['ite_b1b0_short_true']) ** 2) * np.mean(a0pi0_short ** 2)
Ca0_square = np.mean(diff_a0pi0 ** 2) / np.mean(a0pi0_short ** 2)

C1b1_square_S1_square_Ca0_square = C1b1_square * S1_square * Ca0_square  # equals B1_square_ub 


Cy_square = np.mean(diff_b1 ** 2) / np.mean((predict_test['Y'] - predict_test['b1_short']) ** 2)
S2_square = np.mean((predict_test['Y'] - predict_test['b1_short']) ** 2) * np.mean((a1pi1_short * a0pi0_short) ** 2)
Ca0a1_square = (np.mean((a1pi1_true * a0pi0_true - a1pi1_short * a0pi0_short) ** 2) / 
                np.mean((a1pi1_short * a0pi0_short) ** 2))  # equals B2_square_ub

Cy_square_S2_square_Ca0a1_square = Cy_square * S2_square * Ca0a1_square

display(Math(r"C_{{b_1^*}}^2 S_1^2 C_{{A_0}}^2 = {:.4f}".format(C1b1_square_S1_square_Ca0_square)))
display(Math(r"C_{{Y}}^2 S_2^2 C_{{\bar{{A}}}}^2 = {:.4f}".format(Cy_square_S2_square_Ca0a1_square)))



<IPython.core.display.Math object>

<IPython.core.display.Math object>

### 2.9 Summary table

In [18]:

# Define the quantities to include in the summary table
summary_data = {
    "Row Name": [
        r"$\mathbb{E}[Y^{\bar{1}}]$",
        r"$\psi$",
        r"$\psi^*$",
        r"$\psi^* - \psi$",
        r"Basic OVB",
        r"$B_1 + B_2$",
        r"$|\psi^* - \psi| \leq |B_1| + |B_2| \leq (C_{{b_1^*}}^2 S_1^2 C_{{A_0}}^2)^{1/2} + (C_{{Y}}^2 S_2^2 C_{{\bar{{A}}}}^2)^{1/2}$"
    ],
    "Value": [
        mean_Y_bar1,  
        psi_true,  
        psi_short,  
        diff_psi,  
        ovb_basic,  
        sum_B1_B2,  
        ovb_ub  
    ]
}

# Create the table header with spacing
markdown_table = "| Row Name                              | Value     |\n"
markdown_table += "|---------------------------------------|-----------|\n"

# Add each row to the table with better spacing
for name, value in zip(summary_data["Row Name"], summary_data["Value"]):
    markdown_table += f"| {name}                                | {value:.6f} |\n"

# Display the formatted table
display(Markdown(markdown_table))


| Row Name                              | Value     |
|---------------------------------------|-----------|
| $\mathbb{E}[Y^{\bar{1}}]$                                | 1.519582 |
| $\psi$                                | 1.520553 |
| $\psi^*$                                | 1.654579 |
| $\psi^* - \psi$                                | 0.134026 |
| Basic OVB                                | 0.133919 |
| $B_1 + B_2$                                | 0.133792 |
| $|\psi^* - \psi| \leq |B_1| + |B_2| \leq (C_{{b_1^*}}^2 S_1^2 C_{{A_0}}^2)^{1/2} + (C_{{Y}}^2 S_2^2 C_{{\bar{{A}}}}^2)^{1/2}$                                | 0.346964 |
